In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import re
import datetime as dt

In [ ]:
df = pd.read_csv('Reformation_table.csv')

In [ ]:
df.rename({'brand' : 'brand_name'}, axis=1, inplace=True)
df.rename({'product_name' : 'display_name'}, axis=1, inplace=True)
df.rename({'url' : 'product_url'}, axis=1, inplace=True)
df.rename({'available' : 'size'}, axis=1, inplace=True)

In [ ]:
df.color = df.color.str.replace(']',"").str.replace('[',"").str.replace("'","").str.split(', ')

In [ ]:
df.image = df.image.str.replace(']',"").str.replace('[',"").str.replace("'","").str.split(', ')

In [ ]:
GranCat = pd.read_csv("LowLevelCatagoryRegex.csv", header=None, skiprows=[0])

def process_cat(df):
    # get all the values in category regex
    subCats = GranCat.iloc[:, 1:]
    Cats = []
    for index, row in subCats.iterrows():
        for i in GranCat.columns[1:]:
            if pd.isnull(row[i]): continue
            else: Cats.append(row[i])
               
    # assign potential category for each clothing row based on title, else other
    clothesCat = []
    for index, row in df.iterrows():
        rowCat = float("NaN")
        for cat in Cats:
            if re.search(cat, row['display_name'].lower()):
                rowCat = cat
                print(rowCat)
                break
        if pd.isnull(rowCat):
            clothesCat.append('other')
        else:
            clothesCat.append(rowCat)

    return clothesCat

# set parent category to each product
def get_cat(x):
    if x == 'other': return x
    else: return [GranCat[0][i] for i in GranCat.index if x in GranCat.iloc[i].to_list()][0]

# get low_level column given a df
def get_lowlevel(df):
    df['low_level_new'] = process_cat(df)
    df['low_level_new'] = df['low_level_new'].apply(lambda x: get_cat(x))
    return df
# get unique id
def get_id(df, date='20210901'):
    brand = df['brand_name'][0]
    clothing_ids = []
    for i in range(len(df.index)):
        clothing_id = date + brand + str(i)
        clothing_ids.append(clothing_id)
    return clothing_ids
def get_dic_image(row):
    color_image_dict = {'color': 'image_url'}
    for i in range(len(row['color'])):
        #print(i)
        #{row['color'][i]: row['image'][i]}
        try:
            color_image_dict = {row['color'][i]:row['image'][i]}
        except:
             color_image_dict = {row['color'][i]:row['image'][0]}
            

        
    return color_image_dict

def get_price(x): 
    try: return x.split(': ')[1]
    except: return 'Unknown'

def change_gender(x):
    if "Men" in x and "women" not in x.lower(): return 'Men'
    else: return 'Women'

In [ ]:
##### get mapped_id
GranRegs = pd.read_csv('MaterialProxy.csv', header=None, skiprows=[0])
GranRegs = GranRegs.drop(columns=[1])
GranRegs.columns = list(range(len(GranRegs.columns)))

In [ ]:
df['fabric'] = df['fabric'].fillna('')

In [ ]:
#extract materials

def extractMaterials(df):
    fabric_text = df['fabric']
    fabric_text = str(fabric_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_fabric = fabric_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '').split()
    pattern = "\d{1,3}[$%]"
    
    description_text = df['description'] + df['display_name'] + df['fabric']
    description_text = str(description_text).replace('solvent', '').replace('Grown', '').replace('grown', '').replace('organically', 'organic').replace('Organically', 'organic')
    splitted_description = description_text.replace("'", '').replace("[", '').replace("]", '').replace(",", '').replace('’', '').replace('.', '').replace('–', '').replace('-', '')
    pattern = "\d{1,3}[$%]"
    
    mat_list = []
    extra_materials = []
    
    
    ## getting the materials regex matching
    subRegs = GranRegs.iloc[:, 1:]
    Regs = []
    materialsReg = []
    for index, row in subRegs.iterrows():
        for i in range(1,len(GranRegs.columns)):
            if pd.isnull(row[i]): continue
            else: Regs.append(row[i])
    total_percentage = [] 
    is_no_percentage = True 
    for i in range(len(splitted_fabric)):
        if re.fullmatch(pattern,splitted_fabric[i]):
            total_percentage.append(float(splitted_fabric[i].replace("%", '')))
            if sum(total_percentage) <= 100:
            
                if splitted_fabric[i+1].lower() == 'organic':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                elif splitted_fabric[i+1].lower() == 'recycled':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                
                elif splitted_fabric[i+1].lower() == 'tencel™':
                    mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() + ' ' + splitted_fabric[i+2].lower())
                    is_no_percentage = False
                    
                else:
                    if splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower() not in mat_list:
                        mat_list.append(splitted_fabric[i] + ' ' + splitted_fabric[i+1].lower())
                        is_no_percentage = False
                        
    if sum(total_percentage) < 100 or is_no_percentage == True:
        for reg in Regs:
            #print(splitted_description.lower())
            if re.search(reg, splitted_description.lower()):
                match = re.search(reg, splitted_description.lower())
                matched_material = splitted_description[match.start():match.end()].lower()
                
                if matched_material not in extra_materials:
                    extra_materials.append(matched_material)
    
    #print(extra_materials)
    if len(extra_materials)>1:
        if "cotton" in extra_materials and "organic cotton" in extra_materials:
            mat_list.append("100% organic cotton")
        if "leather" in extra_materials:
            mat_list.append("100% leather")
    elif len(extra_materials)!=0:
        mat_list.append("100% " + extra_materials[0])

            
    
    return mat_list



In [ ]:
df['materials'] = df.apply(lambda x: extractMaterials(x), axis=1)

In [ ]:
df[df["materials"].str.len() == 0]['fabric']

52                            
64          ['Dry clean only']
88          ['Dry clean only']
138    ['Wash cold\nLine dry']
Name: fabric, dtype: object

In [ ]:
#df.insert(loc=0, column='clothing_id', value=get_id(df,'20210825'))
#df['material_dict'] = ''
#df['overallscore'] = 4.5
df['color'] = df['color'].fillna('Unknown')
#df = df.drop(columns=['Unnamed: 0','product_material'])
df['image_link_color'] = df.apply(lambda x: get_dic_image(x), axis=1)
df = get_lowlevel(df)
df['gender'] = df['display_name'].apply(lambda x: change_gender(x))
df['scrapped_date'] = dt.date.today()

dresse?s?$
dresse?s?$
jeans
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
jeans
dresse?s?$
loafers?
tops?
dresse?s?$
dresse?s?$
jeans
pants?
tops?
two piece
dresse?s?$
tops?
dresse?s?$
blazers?
dresse?s?$
sk[io]rts?
dresse?s?$
dresse?s?$
dresse?s?
dresse?s?$
tops?
jeans
bustier
dresse?s?$
jeans
jeans
dresse?s?$
dresse?s?$
jeans
dresse?s?$
dresse?s?
dresse?s?$
two piece
dresse?s?$
dresse?s?$
sk[io]rts?
sk[io]rts?
dresse?s?
bra
sk[io]rts?
tops?
jeans
dresse?s?$
dresse?s?$
tops?
dresse?s?$
jeans
jackets?
jeans
tops?
sk[io]rts?
jackets?
dresse?s?$
dresse?s?
dresse?s?$
dresse?s?$
blazers?
pants?
dresse?s?$
tops?
dresse?s?$
tops?
dresse?s?
tops?
dresse?s?$
two piece
dresse?s?$
dresse?s?
dresse?s?$
sk[io]rts?
dresse?s?$
two piece
dresse?s?
dresse?s?$
jackets?
bra
two piece
tops?
dresse?s?$
dresse?s?$
dresse?s?$
dresse?s?$
two piece
dresse?s?$
dresse?s?$
pants?
dresse?s?$
sweaters?
jeans
tops?
jeans
tops?
tops?
sk[io]rts?
jeans
tops?
dresse?

In [ ]:
df[['display_name','low_level_new']]

,display_name,low_level_new
0,Bard Dress,Midi Dresses
1,Lloyd Dress,Midi Dresses
2,Liza Ultra High Rise Straight Jeans,Jeans
3,Fulton Dress,Midi Dresses
4,Karah Dress,Midi Dresses
5,Enid Dress,Midi Dresses
6,Gavin Dress,Midi Dresses
7,Alessi Dress,Midi Dresses
8,Murietta Dress,Midi Dresses
9,Benedict Dress,Midi Dresses


In [ ]:
df[df['low_level_new']=='other'][['display_name','low_level_new']]

,display_name,low_level_new
19,Abalonia Chunky Maryjane,other
80,Nashville Linen Short Es,other
117,Charlie High Rise Jean Short,other
126,Kayo High &amp; Skinny,other
193,Cashmere Sweatsuit,other
195,Cashmere Relaxed V-Neck,other


In [ ]:
df = df.rename(columns={"low_level_new": "low_level"})

In [ ]:
df = df.rename(columns={"id": "clothing_id"})

In [ ]:
new_df_materials = []
for i in range(len(df.materials)):
    material_dict={}
    for percentage_material in df['materials'].iloc[i]:
        percentmat_list = percentage_material.split('% ')
        for i in range(0,len(GranRegs.index)):
            if percentmat_list[1] in GranRegs.iloc[i].to_list():
                new_mat_id = GranRegs[0][i]
        #print(new_mat_id)
        material_dict.update({new_mat_id:percentmat_list[0] + "%"})
    new_df_materials.append(material_dict)
df.materials = pd.Series(new_df_materials)

In [ ]:
df.drop(df[df["materials"].str.len() == 0]['fabric'].index, inplace=True)

In [ ]:
df = df[df['low_level'] != 'other']

#### Push to DB

In [ ]:
df['color'] = df['color'].apply(lambda x: str(x))

In [ ]:
def formatDFforDB(df):
    
    
    clothing_df_copy = df.copy()
    clothing_df_copy['image_link_color'] = clothing_df_copy['image_link_color'].apply(json.dumps)
    clothing_df_copy['material_dict'] = clothing_df_copy['material_dict'].apply(json.dumps)
    clothing_df_copy['color'] = clothing_df_copy['color'].str.replace('[','{').str.replace(']','}')
    clothing_df_copy['size'] = clothing_df_copy['size'].str.replace('[','{').str.replace(']','}')
    clothing_df_copy['size'] = clothing_df_copy['size'].str.replace('"','inches')

    return clothing_df_copy